# IBM_Data Science Capstone Project

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
# Get the link
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data1 = requests.get(url).text

In [88]:
#data1

In [3]:
# Scrape data from that link using BeautifulSoup
data = BeautifulSoup(data1, 'xml')

In [4]:
data_new = data.find('table')
#data_new

In [5]:
# Create an empty dataframe with three columns:
columns = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns=columns)
df

,PostalCode,Borough,Neighborhood


In [6]:
# get data for PostalCode, Borough, Neighborhood
for tr in data_new.find_all('tr'):
    empty_df=[]
    for td in tr.find_all('td'):
        empty_df.append(td.text.strip())
    if len(empty_df) ==3:
        df.loc[len(df)] = empty_df

In [7]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
# find all Borough that are "Not assigned" and remove them
df = df[df['Borough'] !='Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
# groupby Postalcode
df_group = df.groupby(["PostalCode","Borough"])['Neighborhood'].apply(lambda x: ','.join(x))
df_group = df_group.reset_index(drop=False)
df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df.shape

(103, 3)

## Part 2

In [17]:
# get latitude and longitude from csv file
geo_data = pd.read_csv("E:\TANHTRAN\Education\Coursera\IBM Data Science\Course9_Capstone\Week 3\Geospatial_Coordinates.csv")

In [18]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
# merge geo_data with df based on column "PostalCode"
# But we need to change the name of the column "Postal Code" of geo_data to "PostalCode" to match with the name in df in order to merge
geo_data.rename(columns={'Postal Code':"PostalCode"}, inplace=True) # inplace = True to replace that for later use
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
# Check again data info of df:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
# Now, lets merge geo_data with df, and named as df_new
df_new = pd.merge(geo_data, df, on = "PostalCode") # merge 2 dataframes based on column "PostalCode"
df_new.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [25]:
# We can change the order of the columns
df_new = df_new[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
